# ***Assignment: Ad Image Insertion in Video with Occlusion Handling***

**Importing Libraries**

In [ ]:
pip install opencv-python matplotlib

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from google.colab.patches import cv2_imshow

*The necessary libraries imported are:*

`cv2` for computer vision tasks<br>
`numpy` for numerical computations

**Loading Necessary Files**

In [ ]:
image = "advertisement image.jpg"
video = "input video.mp4"
foreground_path = "advertisement image.jpg"
foreground = cv2.imread(foreground_path)
cap = cv2.VideoCapture('input video.mp4')

**Extraction Of Object From Image**

In [ ]:
# Load the image
image = cv2.imread(image)  # Replace with the path to your image
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

# Convert the image to grayscale
gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)

# Perform thresholding to create a binary mask (adjust threshold values as needed)
_, thresh = cv2.threshold(gray, 200, 255, cv2.THRESH_BINARY)

# Apply the mask to the original image to extract the object
extracted_object = cv2.bitwise_and(image, image, mask=thresh)

# Display the original image and the extracted object using Matplotlib
fig, axes = plt.subplots(1, 2, figsize=(12, 6))

axes[0].imshow(image)
axes[0].set_title('Original Image')
axes[0].axis('off')

axes[1].imshow(extracted_object)
axes[1].set_title('Extracted Object')
axes[1].axis('off')

plt.tight_layout()
plt.show()

# Get the resolution of the image
height, width, channels = image.shape

print(f"Image Resolution: Width = {width}, Height = {height}, Channels = {channels}")

In [ ]:
# Check if the foreground image is loaded successfully
if foreground is None:
    print("Error: Unable to load the foreground image.")
else:
    # Open the video file (change 'input_video2.mp4' to your video file name)
    cap = cv2.VideoCapture('input video.mp4')

    # Check if the video file is opened successfully
    if not cap.isOpened():
        print("Error: Unable to open the video file.")
    else:
        # Get the original dimensions of the video
        original_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
        original_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

        # Set the desired dimensions for the resized video
        output_width = 640  # Set your desired width
        output_height = 480  # Set your desired height

        # Create VideoWriter object to write the output video
        fourcc = cv2.VideoWriter_fourcc(*'XVID')
        out = cv2.VideoWriter('output_video.avi', fourcc, 20.0, (output_width, output_height))

        # Set initial value of weights
        alpha = 0.4

        while True:
            # read the background
            ret, background = cap.read()

            if not ret:
                break  # Break the loop if we reach the end of the video



            # Resize the foreground image to match the dimensions of the region in the background
            foreground_resized = cv2.resize(foreground, (200, 200))

            # Select the region in the background where we want to add the image and add the images using cv2.addWeighted()
            added_image = cv2.addWeighted(
                background[original_height-300:original_height-100, original_width-300:original_width-100, :],
                alpha,
                foreground_resized,
                1 - alpha,
                0
            )


            # Change the region with the result
            background[original_height-300:original_height-100, original_width-300:original_width-100] = added_image

            # Resize the output frame to fit the desired dimensions
            output_frame = cv2.resize(background, (output_width, output_height))

            # Write the frame to the output video file
            out.write(output_frame)

            # For displaying the current value of alpha(weights)
            font = cv2.FONT_HERSHEY_SIMPLEX
            cv2.putText(output_frame, 'Press "a" to increase alpha, "d" to decrease alpha, "q" to quit', (10, 30),
                        font, 0.7, (255, 255, 255), 2, cv2.LINE_AA)
            cv2.putText(output_frame, 'Current alpha: {{{{{{{{:.2f}}}}}}}}'.format(alpha), (10, 60),
                        font, 0.7, (255, 255, 255), 2, cv2.LINE_AA)
            cv2_imshow(output_frame)

            k = cv2.waitKey(10)  # Wait for 10 milliseconds

            # Press q to break
            if k == ord('q'):
                break

            # Press a to increase alpha by 0.1
            elif k == ord('a'):
                alpha += 0.1
                if alpha >= 1.0:
                    alpha = 1.0

            # Press d to decrease alpha by 0.1
            elif k == ord('d'):
                alpha -= 0.1
                if alpha <= 0.0:
                    alpha = 0.0

# Release the video file and destroy all windows
cap.release()
out.release()
cv2.destroyAllWindows()

In [ ]:
import os

**Application Of The Cascade library for graceful Occlusion handling**

In [ ]:
def detect_hand(video_path, output_path, base_hand_level=0.3, image_scale=0.7):

    haar_cascade_path = os.path.abspath('haarcascade_hand.xml')

    hand_cascade = cv2.CascadeClassifier(haar_cascade_path)

    top_left_y = 600
    top_left_x = 120

    # Get video properties
    video = cv2.VideoCapture(video_path)
    width = int(video.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(video.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = video.get(cv2.CAP_PROP_FPS)

    # Define VideoWriter
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out_writer = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

    while True:
        ret, frame = video.read()

        if not ret:
            break

        # Convert the frame to grayscale for hand detection
        gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        # Detect hands using the cascade classifier
        hands = hand_cascade.detectMultiScale(gray_frame, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

        if len(hands) > 0:
            # Assuming only one hand is detected, take the first hand
            x, y, w, h = hands[0]

            # Check if the hand is above a certain level
            if y < base_hand_level * height:
                # Move the image when the hand is above the specified level
                top_left_y = max(0, y - int(image_scale * height))


            resized_image = cv2.resize(image, (int(image_scale * w), int(image_scale * h)))


            alpha_channel = cv2.resize(resized_image[:, :, 2], (resized_image.shape[1], resized_image.shape[0])) / 255.0
            frame[top_left_y:top_left_y + resized_image.shape[0], top_left_x:top_left_x + resized_image.shape[1]] = (
                    (1 - alpha_channel)[:, :, np.newaxis] * frame[top_left_y:top_left_y + resized_image.shape[0], top_left_x:top_left_x + resized_image.shape[1]]
                    + alpha_channel[:, :, np.newaxis] * resized_image[:, :, :3]
            )

        out_writer.write(frame)

        cv2_imshow( frame)
        if cv2.waitKey(25) & 0xFF == ord('q'):
            break

    video.release()
    out_writer.release()
    cv2.destroyAllWindows()

video_path = "input video.mp4"
output_path = "output_video_with_logo.mp4"

detect_hand(video_path, output_path)
